<a href="https://colab.research.google.com/github/duongdqq/TF_guide/blob/master/3_train_n_evaluation_w_builtin_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# API overview: a first end-to-end example

In [2]:
inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [4]:
print(len(x_train))
print(len(x_test))
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

60000
10000


In [5]:
print(len(x_train))
print(len(x_val))
print(len(x_test))
print('-'*10)
print(len(y_train))
print(len(y_val))
print(len(y_test))

50000
10000
10000
----------
50000
10000
10000


In [6]:
# specify the training configuration
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

In [7]:
# call fit() to slide data into batches
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    validation_data=(x_val, y_val)
) 

Epoch 1/2
782/782 [==============================] - 2s 2ms/step - loss: 0.3347 - sparse_categorical_accuracy: 0.9051 - val_loss: 0.1789 - val_sparse_categorical_accuracy: 0.9509
Epoch 2/2
782/782 [==============================] - 2s 2ms/step - loss: 0.1563 - sparse_categorical_accuracy: 0.9534 - val_loss: 0.1349 - val_sparse_categorical_accuracy: 0.9610


In [8]:
# history holds record of the loss values and metric values during training
history.history

{'loss': [0.3346588611602783, 0.15630881488323212],
 'sparse_categorical_accuracy': [0.9051200151443481, 0.9534000158309937],
 'val_loss': [0.1788896918296814, 0.13488304615020752],
 'val_sparse_categorical_accuracy': [0.9509000182151794, 0.9610000252723694]}

In [9]:
# evaluate model on the test data
results = model.evaluate(x_val, y_val, batch_size=128)

79/79 [==============================] - 0s 1ms/step - loss: 0.1349 - sparse_categorical_accuracy: 0.9610


In [10]:
predictions = model.predict(x_test[:3])
predictions

array([[5.9501331e-06, 1.3490105e-08, 7.0690113e-04, 1.1235866e-03,
        2.5489769e-07, 3.5253506e-06, 2.7892084e-09, 9.9810767e-01,
        4.1798708e-06, 4.7927464e-05],
       [4.3341752e-06, 1.1717662e-05, 9.9866831e-01, 1.2066624e-03,
        1.0976159e-11, 3.9320603e-05, 4.8560392e-05, 8.4636156e-09,
        2.1136615e-05, 2.7921901e-10],
       [4.1748335e-05, 9.9112260e-01, 1.2611914e-03, 3.6432597e-04,
        4.1625666e-04, 1.4570988e-04, 9.1897859e-04, 4.6847328e-03,
        8.8739785e-04, 1.5701985e-04]], dtype=float32)

In [11]:
predictions.shape

(3, 10)

# The compile() method: specifying a loss, metrics, and an optimizer

* Specify optimizer, loss function and metrics by model.compile()

In [12]:
inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

* 1st method

In [13]:
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

* If model has multiple outputs, specify different losses and metrics for each output, and modulate the contribution of each output to the total loss of the model

* 2nd method

In [14]:
model.compile(
    optimizer='rmsprop',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

* For later reuse, put the model definition and compile step in function

In [15]:
def get_uncompiled_model():
  inputs = keras.Input(shape=(784,), name='digits')
  x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
  x = layers.Dense(64, activation='relu', name='dense_2')(x)
  outputs = layers.Dense(10, activation='softmax', name='predictions')(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model

In [16]:
def get_compiled_model():
  model = get_uncompiled_model()
  model.compile(
    optimizer='rmsprop',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
  )
  return model

### Many built-in optimizers, losses, and metrics are available

* Optimizers
  * SDG() with or without momentum
  * RMSprop()
  * Adam()
* Losses
  * MeanSquaredError()
  * KLDivergence()
  * CosineSimilarity()
* Metrics
  * AUC()
  * Precision()
  * Recall()

### Custom losses

* 1st method

In [17]:
def custom_mean_squared_error(y_true, y_pred):
  return tf.math.reduce_mean(tf.square(y_true - y_pred))

model = get_uncompiled_model()
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=custom_mean_squared_error
)

In [18]:
y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)

782/782 [==============================] - 2s 2ms/step - loss: 0.0159


* 2nd method

* __init__(self): accept parameters to pass during the call of your loss function
* call(self, y_true, y_pred): use the targets (y_true) and the model predictions (y_pred) to compute the model's loss

In [19]:
class CustomMSE(keras.losses.Loss):
  def __init__(self, regularization_factor=0.1, name='custom_mse'):
    super().__init__(name=name)
    self.regularization_factor = regularization_factor

  def call(self, y_true, y_pred):
    mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
    reg = tf.math.reduce_mean(tf.square(0.5 - y_pred))
    return mse + reg * self.regularization_factor


model = get_compiled_model()
model.compile(optimizer=keras.optimizers.Adam(),
              loss=CustomMSE())

y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)

782/782 [==============================] - 2s 2ms/step - loss: 0.0386


### Custom metrics

* __init__(self), in which you will create state variables for your metric.
* update_state(self, y_true, y_pred, sample_weight=None), which uses the targets * y_true and the model predictions y_pred to update the state variables.
* result(self), which uses the state variables to compute the final results.
* reset_states(self), which reinitializes the state of the metric.

* State update and results computation are kept separate (in update_state() and result(), respectively) because in some cases, results computation might be very expensive, and would only be done periodically.

In [20]:
class CategoricalTruePositives(keras.metrics.Metric):
  def __init__(self, name='categorical_true_positives', **kwargs):
    super(CategoricalTruePositives, self).__init__(name=name, **kwargs)
    self.true_positives = self.add_weight(name='ctp', initializer='zeros')

  
  def update_state(self, y_true, y_pred, sample_weight=None):
    y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1, 1))
    values = tf.cast(y_true, 'int32') == tf.cast(y_pred, 'int32')
    values = tf.cast(values, 'float32')
    if sample_weight is not None:
      sample_weight = tf.cast(sample_weight, 'float32')
      values = tf.multiply(values, sample_weight)
    self.true_positives.assign_add(tf.reduce_sum(values))

  
  def result(self):
    return self.true_positives

  
  def reset_states(self):
  # The state of the metric will be reset at the start of each epoch.
    self.true_positives.assign(0.0)

In [21]:
model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[CategoricalTruePositives()])

model.fit(x_train, y_train, batch_size=128, epochs=3)


Epoch 1/3
391/391 [==============================] - 1s 3ms/step - loss: 0.3977 - categorical_true_positives: 44524.0000
Epoch 2/3
391/391 [==============================] - 1s 3ms/step - loss: 0.1878 - categorical_true_positives: 47257.0000
Epoch 3/3
391/391 [==============================] - 1s 3ms/step - loss: 0.1425 - categorical_true_positives: 47897.0000


### Handling losses and metrics that don't fit the standard signature

* The overwhelming majority of losses and metrics can be computed from y_true and y_pred, where y_pred is an output of your model. 
* But not all of them. For instance, a regularization loss may only require the activation of a layer (there are no targets in this case), and this activation may not be a model output.

* In such cases, you can call self.add_loss(loss_value) from inside the call method of a custom layer. 
* Losses added in this way get added to the "main" loss during training (the one passed to compile()). 
* Here's a simple example that adds activity regularization (note that activity regularization is built-in in all Keras layers -- this layer is just for the sake of providing a concrete example):

In [22]:
class ActivityRegularizationLayer(layers.Layer):
  def call(self, inputs):
    self.add_loss(tf.reduce_sum(inputs) * 0.1)
    return inputs  # pass through layer


inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)

# insert activity regurlarization as a layer
x = ActivityRegularizationLayer()(x)

x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True))

model.fit(x_train, y_train, batch_size=128, epochs=1)

391/391 [==============================] - 1s 3ms/step - loss: 3.1455


* You can do the same for logging metric values, using add_metric():

In [23]:
class MetricLoggingLayer(layers.Layer):
  def call(self, inputs):
    self.add_metric(keras.backend.std(inputs), 
                    name='std_of_activation', 
                    aggregation='mean')
    return inputs


inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)

# Insert std logging as a layer.
x = MetricLoggingLayer()(x)

x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)
model.fit(x_train, y_train, batch_size=64, epochs=1)

782/782 [==============================] - 2s 2ms/step - loss: 0.3462 - std_of_activation: 1.0496


* In the Functional API, you can also call 
* model.add_loss(loss_tensor), or 
* model.add_metric(metric_tensor, name, aggregation).

In [24]:
inputs = keras.Input(shape=(784,), name='digits')
x1 = layers.Dense(64, 'relu', name='dense1')(inputs)
x2 = layers.Dense(64, 'relu', name='dense2')(x1)
outputs = layers.Dense(10, name='predictions')(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

model.add_loss(tf.reduce_sum(x1) * 0.1)
model.add_metric(keras.backend.std(x1), name='std_of_activation', aggregation='mean')

model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True))
model.fit(x_train, y_train, batch_size=256, epochs=1)

196/196 [==============================] - 1s 4ms/step - loss: 5.7530 - std_of_activation: 0.0073


* Note that when you pass losses via add_loss(), it becomes possible to call compile() without a loss function, since the model already has a loss to minimize.
* Consider the following LogisticEndpoint layer: it takes as inputs targets & logits, and it tracks a crossentropy loss via add_loss(). It also tracks classification accuracy via add_metric().

In [25]:
class LogisticEndpoint(keras.layers.Layer):
  def __init__(self, name=None):
    super(LogisticEndpoint, self).__init__(name=name)
    self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
    self.accuracy_fn = keras.metrics.BinaryAccuracy()

  def call(self, targets, logits, sample_weights=None):
    # Compute the training-time loss value and add it
    # to the layer using `self.add_loss()`.
    loss = self.loss_fn(targets, logits, sample_weights)
    self.add_loss(loss)

    # Log accuracy as a metric and add it
    # to the layer using `self.add_metric()`.
    acc = self.accuracy_fn(targets, logits, sample_weights)
    self.add_metric(acc, name='accuracy')

    # Return the inference-time prediction tensor (for `.predict()`).
    return tf.nn.softmax(logits)

* You can use it in a model with two inputs (input data & targets), compiled without a loss argument, like this:

In [26]:
import numpy as np

inputs = keras.Input(shape=(3,), name='inputs')
targets = keras.Input(shape=(10, ), name='targets')
logits = keras.layers.Dense(10)(inputs)
predictions = LogisticEndpoint(name='pred')(logits, targets)

model = keras.Model(inputs=[inputs, targets], outputs=predictions)
model.compile(optimizer='adam')  

data = {'inputs': np.random.random((3,3)),
        'targets': np.random.random((3,10))}

model.fit(data)

1/1 [==============================] - 0s 2ms/step - loss: 0.9617 - binary_accuracy: 0.0000e+00


### Automatically setting apart a validation holdout set

* Here's another option: the argument validation_split allows you to automatically reserve part of your training data for validation. The argument value represents the fraction of the data to be reserved for validation, so it should be set to a number higher than 0 and lower than 1. For instance, validation_split=0.2 means "use 20% of the data for validation", and validation_split=0.6 means "use 60% of the data for validation".

In [27]:
model = get_compiled_model()
model.fit(x_train, y_train, batch_size=64, validation_split=0.2, epochs=1)

625/625 [==============================] - 2s 3ms/step - loss: 0.3709 - sparse_categorical_accuracy: 0.8964 - val_loss: 0.2263 - val_sparse_categorical_accuracy: 0.9320


# Training & evaluation from tf.data Datasets

## Using a validation dataset

# Other input formats supported

# Using a keras.utils.Sequence object as input

# Using sample weighting and class weighting

## Class weights

## Sample weights

# Passing data to multi-input, multi-output models

# Using callbacks

## Many built-in callbacks are available

## Writing your own callback

# Checkpointing models


# Using learning rate schedules

## Passing a schedule to an optimizer

## Using callbacks to implement a dynamic learning rate schedule

# Visualizing loss and metrics during training

## Using the TensorBoard callback